# Stored procedures in Azure Cosmos DB

Let's have some fun with [stored procedures](https://docs.microsoft.com/azure/cosmos-db/stored-procedures-triggers-udfs)! They let you implement custom logic in Javascript functions that get directly executed on your Cosmos DB instance. Stored procedures are most useful when you need to run atomic transactions over multiple items.

We start by creating the resources we are going to need: a database and a container.

In [1]:
# Initialize the client (not required if run on the user's account?)
import azure.cosmos
import azure.cosmos.cosmos_client as cosmos_client
import time
client = cosmos_client.CosmosClient(os.environ["COSMOS_ENDPOINT"], {'masterKey': os.environ["COSMOS_KEY"]})

db_name    = "spdbtest"
container_name  = "spcltest"
db_link = "/dbs/" + db_name
container_link = "/dbs/" + db_name + "/colls/" + container_name

# Create the database if it doesn't exist
db_query = "select * from r where r.id = '{0}'".format(db_name)
db = list(client.QueryDatabases(db_query))
if db:
    print('Database already exists')
else:
    client.CreateDatabase({'id': db_name})
    print('Database created')
    time.sleep(3)

# Reset the container
container_query = "select * from r where r.id = '{0}'".format(container_name)
container = list(client.QueryContainers(db_link, container_query))
if container:
    client.DeleteContainer(container_link)
    print('Container dropped')
client.CreateContainer(db_link, {'id': container_name})
print('Container created')

Database already exists
Container dropped
Container created


Now that we have a container to work with, we can create a stored procedure named `createItem` that just stores any object passed as a parameter. It's nothing more that a Javascript function that uses [Cosmos DB's server-side API](https://azure.github.io/azure-cosmosdb-js-server/) to interact with the container it's running in:

In [2]:
sp1_name = 'createItem'
sp1_link = container_link + "/sprocs/" + sp1_name
sp1_definition = {
    'id': sp1_name,
    'serverScript': """function (itemToCreate) {

    var context = getContext();
    var container = context.getCollection();

    // create the new item
    container.createDocument(container.getSelfLink(),
        itemToCreate,
        function (err, itemCreated) {
            if (err) throw new Error('Error' + err.message);
            context.getResponse().setBody(itemCreated.id)
        });
}"""
}
client.CreateStoredProcedure(container_link, sp1_definition)
print('Stored procedure "' + sp1_name + '" created')

Stored procedure "createItem" created


Can't wait to try it! Let's call it by passing some JSON object as a parameter:

In [3]:
result = client.ExecuteStoredProcedure(sp1_link, {
    'name': 'Alice'
})
print('New item with id "' + result + '" created')

New item with id "b9096dfc-c242-0008-b1ad-3417bcd3629d" created


Looks like our stored procedure has created an item based on the JSON object we passed. We can verify that by reading that new item and looking up the value of its `name` property:

In [4]:
new_item = client.ReadItem(container_link + "/docs/" + result)
print('"name" value is: ' + new_item['name'])

"name" value is: Alice


Great! But not super useful... We could have done that directly client-side without the need for a stored procedure. Let's spice things up with a second one that creates an item only if a pre-condition is met:

In [5]:
sp2_name = 'createItemConditional'
sp2_link = container_link + "/sprocs/" + sp2_name
sp2_definition = {
    'id': sp2_name,
    'serverScript': """function (nameValueToCheck, itemToCreate) {

    var context = getContext();
    var container = context.getCollection();
    
    var query = 
    {     
        'query' : 'SELECT * FROM c where c.name = @val',
        'parameters' : [{ 'name': '@val', 'value': nameValueToCheck }] 
    };
    // check for the pre-condition
    container.queryDocuments(container.getSelfLink(),
        query,
        function (err, items) {
            if (err) throw new Error('Error' + err.message);
            if (items.length > 0) {
                // if no item matches the pre-condition, create the new item
                container.createDocument(container.getSelfLink(),
                    itemToCreate,
                    function (err, itemCreated) {
                        if (err) throw new Error('Error' + err.message);
                        context.getResponse().setBody(itemCreated.id);
                    });            
            }
            else {
                context.getResponse().setBody('no item created');
            }
        });
}"""
}
client.CreateStoredProcedure(container_link, sp2_definition)
print('Stored procedure "' + sp2_name + '" created')

Stored procedure "createItemConditional" created


So this second stored procedure first searches for items where the value of the `name` property matches the first parameter. If any such document is found, it creates a new item from the JSON object passed as second parameter; if not, it does nothing.

Let's try it:

In [6]:
result = client.ExecuteStoredProcedure(sp2_link, ['Bob', {
    'name': 'Bob'
}])
print(result)

no item created


Because our container currently doesn't contain any item where `name` equals `Bob`, the stored procedure didn't create any new item, as expected. But if we pass `Alice` as the first parameter, it will match the item we created just before, and a new item should be created:

In [7]:
result = client.ExecuteStoredProcedure(sp2_link, ['Alice', {
    'name': 'Bob'
}])
print('New item with id "' + result + '" created')

New item with id "bbfd472d-702c-e16d-f191-9fd3c8589ec5" created


It did! And just to be sure, let's fetch that new item to check the value of its `name` property:

In [8]:
new_item = client.ReadItem(container_link + "/docs/" + result)
print('"name" value is: ' + new_item['name'])

"name" value is: Bob


What's important to note here is that stored procedures are executed as **atomic transactions**, so there are only 2 situations possible:
- either the Javascript function succeeds and all the write operations it has performed get committed,
- or the Javascript function fails - because it has thrown an exception for example - and all the write operations it has performed get rolled back.

To illustrate that, let's rewrite our second procedure. This time, we invert the order of operations and start by writing the new item. Then, we check for the pre-condition and throw an exception if it's not met:

In [9]:
sp3_name = 'createItemConditionalWithThrow'
sp3_link = container_link + "/sprocs/" + sp3_name
sp3_definition = {
    'id': sp3_name,
    'serverScript': """function (nameValueToCheck, itemToCreate) {

    var context = getContext();
    var container = context.getCollection();

    // create the new item
    container.createDocument(container.getSelfLink(),
        itemToCreate,
        function (err, itemCreated) {
            if (err) throw new Error('Error' + err.message);
            context.getResponse().setBody(itemCreated.id);
            
            var query = 
            {     
                'query' : 'SELECT * FROM c where c.name = @val',
                'parameters' : [{ 'name': '@val', 'value': nameValueToCheck }] 
            };
            // check for the pre-condition
            container.queryDocuments(container.getSelfLink(),
                query,
                function (err, items) {
                    if (err) throw new Error('Error' + err.message);
                    if (items.length == 0) {
                        // if no item matches the pre-condition, throw an error
                        throw new Error('pre-condition not met');
                    }
                });
        });
}"""
}
client.CreateStoredProcedure(container_link, sp3_definition)
print('Stored procedure "' + sp3_name + '" created')

Stored procedure "createItemConditionalWithThrow" created


We try that new stored procedure, this time by encapsulating its call with a try-except block to catch an exception if thrown:

In [10]:
try:
    result = client.ExecuteStoredProcedure(sp3_link, ['Carol', {
        'name': 'Carol'
    }])
    print('New item with id "' + result + '" created')
except azure.cosmos.errors.HTTPFailure as e:
   print('Exception caught!')

Exception caught!


And an exception got caught. We verify that the stored procedure was executed transactionally by fetching all items where `name` equals `Carol`. Although such item *was* written at the beginning of the function, this write operation must have been rolled back because of the exception.

In [11]:
query = 'SELECT * FROM c WHERE c.name = "Carol"'
results = list(client.QueryItems(container_link, query, {'enableCrossPartitionQuery': True}))
print('Got ' + str(len(results)) + ' result(s)')

Got 0 result(s)


Indeed, we found no matching document.

Obviously, calling that last stored procedure with a matching pre-condition will succeed:

In [12]:
try:
    result = client.ExecuteStoredProcedure(sp3_link, ['Bob', {
        'name': 'Carol'
    }])
    print('New item with id "' + result + '" created')
except azure.cosmos.errors.HTTPFailure as e:
   print('Exception caught!')

new_item3 = client.ReadItem(container_link + "/docs/" + result)
print('"name" value is: ' + new_item3['name'])

New item with id "6418c3d0-68b9-8332-35c9-0ce391b895f8" created
"name" value is: Carol


Learn more about [writing](https://docs.microsoft.com/azure/cosmos-db/how-to-write-stored-procedures-triggers-udfs) and [running](https://docs.microsoft.com/azure/cosmos-db/how-to-use-stored-procedures-triggers-udfs) stored procedures in Cosmos DB!

And before we close, don't forget to clean up the resources we've created:

In [13]:
client.DeleteDatabase(db_link)